In [1]:
import json
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
with open('valid.json', 'r') as f1:
    anno1 = json.load(f1)

In [3]:
# Create a data frame
df = pd.DataFrame(columns = ['imgID', 'imgPath', 'dimensions', 'areas', 'labels', 'polygons', 'bboxes'])

# fill the df with entire anno data
for i in range(len(anno1['images'])):
  tempDict = {'imgID': anno1['images'][i]['id'],
            'imgPath':anno1['images'][i]['file_name'],
            'dimensions': (anno1['images'][i]['height'],anno1['images'][i]['width']),
            'areas': [],
            'labels':[],
            'polygons':[],
            'bboxes':[]}
  df = df.append(tempDict, ignore_index=True)

df.set_index('imgID', inplace=True)

for i in range(len(anno1['annotations'])): 
  locIndex = int(anno1['annotations'][i]['image_id'])
  try:
    df.loc[locIndex]['areas'].append(anno1['annotations'][i]['area'])
    df.loc[locIndex]['labels'].append(anno1['annotations'][i]['category_id'])
    df.loc[locIndex]['polygons'].append(anno1['annotations'][i]['segmentation'])
    df.loc[locIndex]['bboxes'].append(anno1['annotations'][i]['bbox'])
  except:
    continue

df.sort_index(inplace=True)

In [5]:
combinedJson = {
    'annotations': [],
    'categories': anno1['categories'],
    'images': [],
    'info': {},
    'licenses': []
}
annoIDCount = 0
for imgID, row in df.iterrows():
    tempImages = {
      "file_name": row['imgPath'],
      "height": row['dimensions'][0],
      "width": row['dimensions'][1],
      "id": imgID,
    }
    combinedJson['images'].append(tempImages)
    for i in range(len(row['labels'])):
      tempAnno = {
        "area": row['areas'][i],
        "bbox": row['bboxes'][i],
        "category_id": row['labels'][i],
        "id": annoIDCount,
        "image_id": imgID,
        "iscrowd": 0,
        "segmentation": [row['polygons'][i]]
      }
      combinedJson['annotations'].append(tempAnno)
      annoIDCount += 1

with open("CombinedCoco.json", "w") as outfile:
    json.dump(combinedJson, outfile)